In [ ]:
import json

import boto3
from botocore.exceptions import ClientError


def invoke_claude_3_with_text(prompt, **bedrock_kwargs):
    """
    Invokes Anthropic Claude 3 Sonnet to run an inference using the input
    provided in the request body.

    :param prompt: The prompt that you want Claude 3 to complete.
    :return: Inference response from the model.
    """

    # Initialize the Amazon Bedrock runtime client
    client = boto3.client(service_name="bedrock-runtime", region_name="us-east-1")

    # Invoke Claude 3 with the text prompt
    model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

    try:
        response = client.invoke_model(
            modelId=model_id,
            body=json.dumps(
                {
                    "anthropic_version": "bedrock-2023-05-31",
                    "max_tokens": 1024,
                    "messages": [
                        {
                            "role": "user",
                            "content": [{"type": "text", "text": prompt}],
                        }
                    ],
                    **bedrock_kwargs,
                }
            ),
        )

        # Process return just the response string
        result = json.loads(response.get("body").read())
        output_list = result.get("content", [])

        assert len(output_list) == 1

        return output_list[0]["text"]

    except ClientError as err:
        print(
            "error: " "Couldn't invoke Claude 3 Sonnet. Here's why: %s: %s",
            err.response["Error"]["Code"],
            err.response["Error"]["Message"],
        )
        raise

In [ ]:
import re


def extract_json_content(input_string: str):  # TODO improve this to be more elegant
    """
    Extract content from strings like <json>\n{content}\n</json>\n
    and return as str
    """
    pattern = re.compile(r"<json>(.*?)</json>", re.DOTALL)
    match = pattern.search(input_string)

    if match:
        return json.dumps(json.loads(match.group(1)))

    else:
        return None